# Overview

- Load .bit file
- Basic loopback program calibration
    - original code from demo
    - time of flight cal
    - readout phase cal
    
- Load real qubit readout data (old data from ZCU111)
    
- Readout mock - averaged signal (as an example)
- Readout mock - singel-shot signal
    - single-shot with tunable guassian noise
    - iterate the single-shot experiment in python loop

In [ ]:
# You should be on the ml branch of https://github.com/GiuseppeDiGuglielmo/qick
!git branch

In [ ]:
!echo -n "git rev:  "; git rev-parse --short HEAD
!echo -n "git date: "; git log -1 --format=%cd

In [ ]:
import subprocess
branch_name = subprocess.getoutput("git rev-parse --abbrev-ref HEAD")
print(branch_name)

In [ ]:
# Uncomment the following lines to cleanup collected results
!rm -f malab_e_state_A_fpga.csv  malab_g_state_A_fpga.csv
!rm -f malab_e_state_X_fpga.csv  malab_e_state_y_fpga.csv
!rm -f malab_g_state_X_fpga.csv  malab_g_state_y_fpga.csv

# Load .bit file

In [ ]:
# Import the QICK drivers and auxiliary libraries
from qick import *
%pylab inline

In [ ]:
# Current directory
!pwd

In [ ]:
# List custom bitstreams
!ls -l ../216/$branch_name/'00000000'

In [ ]:
!ls -l ../216/$branch_name/'20240501'

In [ ]:
# Choose custom bistreams

DATASET_DATE = '00000000'
#DATASET_DATE = '20240501'

# QICK
#CUSTOM_BIT = '../216/{}/qick_216_orig.bit'.format(branch_name)
#HAS_NN = 0

# # QICK + ILAs + syn/imp optimizations
# CUSTOM_BIT = '../216/{}/qick_216_ila.bit'.format(branch_name)
# HAS_NN = 0

# QICK + NN + syn/imp optimizations
#CUSTOM_BIT = '../216/{}/{}/qick_216_nn_285_385.bit'.format(branch_name, DATASET_DATE)
#CUSTOM_BIT = '../216/{}/{}/qick_216_nn_285_385.bit'.format(branch_name, DATASET_DATE)
#CUSTOM_BIT = '../216/{}/{}/qick_216_nn_150_350.bit'.format(branch_name, DATASET_DATE)
#CUSTOM_BIT = '../216/{}/{}/qick_216_nn_150_550.bit'.format(branch_name, DATASET_DATE)
CUSTOM_BIT = '../216/{}/{}/qick_216_nn_150_550_updated.bit'.format(branch_name, DATASET_DATE)
#CUSTOM_BIT = '../216/{}/{}/qick_216_nn_25_745.bit'.format(branch_name, DATASET_DATE)
#CUSTOM_BIT = '../216/{}/{}/qick_216_nn_5_765.bit'.format(branch_name, DATASET_DATE)
#CUSTOM_BIT = '../216/{}/{}/qick_216_nn_1_769.bit'.format(branch_name, DATASET_DATE)
#CUSTOM_BIT = '../216/{}/{}/qick_216_nn_0_770.bit'.format(branch_name, DATASET_DATE)
HAS_NN = True

# Save traces from NN
SAVE_NN_TRACES = True

# Configure channels
GEN_CH = 0
RO_CH = 0

In [ ]:
# Load bitstream with custom overlay
if not 'CUSTOM_BIT' in locals():
    soc = QickSoc()
else:
    import os
    # Normalize path
    CUSTOM_BIT_FULL_PATH = os.path.normpath(os.getcwd() + '/' + CUSTOM_BIT)
    print('Custom bitsream:', CUSTOM_BIT_FULL_PATH)
    
    soc = QickSoc(bitfile=CUSTOM_BIT_FULL_PATH)

soccfg = soc
print(soccfg)

In [ ]:
## Load bitstream with custom overlay
#soc = QickSoc()
## soc = QickSoc("qick_216_0821.bit")
## Since we're running locally on the QICK, we don't need a separate QickConfig object.
## If running remotely, you could generate a QickConfig from the QickSoc:
##     soccfg = QickConfig(soc.get_cfg())
## or save the config to file, and load it later:
##     with open("qick_config.json", "w") as f:
##         f.write(soc.dump_cfg())
##     soccfg = QickConfig("qick_config.json")
#soccfg = soc
#print(soccfg)

In [ ]:
print('Loaded bitstream:', soccfg.bitfile_name)

# NN Classifier Python API

In [ ]:
from pynq import MMIO

def configure_classifier(scaling_factor, trigger_delay):
    mmio_nn = MMIO(soccfg.NN_0.mmio.base_addr, soccfg.NN_0.mmio.length)
    mmio_nn.write(soccfg.NN_0.register_map.scaling_factor.address, scaling_factor)
    mmio_nn.write(soccfg.NN_0.register_map.trigger_delay.address, trigger_delay)

def reset_classifier(deep_reset = False, index_lo = 0, index_hi = 0):
    RESET_HI = 255
    RESET_LO = 0
    mmio_nn = MMIO(soccfg.NN_0.mmio.base_addr, soccfg.NN_0.mmio.length)
    # This sends a "reset pulse"
    mmio_nn.write(soccfg.NN_0.register_map.out_reset.address, RESET_HI)
    mmio_nn.write(soccfg.NN_0.register_map.out_reset.address, RESET_LO)

    if (deep_reset):
        # Reset in software, in hardware may be more efficient (TBD)
        print('WARNING: This may run for some time... use only for debugging')
        entry_count = ((index_hi - index_lo) + 1) * 2
        for i in range(entry_count):
            soccfg.axi_blk_bram_ctrl_0.mmio.array[i] = 0

def get_classifier_prediction_count():
    mmio_nn = MMIO(soccfg.NN_0.mmio.base_addr, soccfg.NN_0.mmio.length)
    prediction_count = mmio_nn.read(soccfg.NN_0.register_map.out_offset.address)
    return prediction_count

def get_classifier_prediction(index = 0):
    WORD_SIZE_BYTE = 4
    WORD_COUNT_PER_PREDICTION = 2
    mmio_bram = MMIO(soccfg.axi_blk_bram_ctrl_0.base_address, soccfg.axi_blk_bram_ctrl_0.size)
    ground_state_logit = mmio_bram.read(index*WORD_COUNT_PER_PREDICTION*WORD_SIZE_BYTE + 0)
    excited_state_logit = mmio_bram.read(index*WORD_COUNT_PER_PREDICTION*WORD_SIZE_BYTE + WORD_SIZE_BYTE)
    return ground_state_logit, excited_state_logit

def get_classifier_predictions(index_lo = 0, index_hi = 0):
    WORD_SIZE_BYTE = 4
    WORD_COUNT_PER_PREDICTION = 2
    mmio_bram = MMIO(soccfg.axi_blk_bram_ctrl_0.base_address, soccfg.axi_blk_bram_ctrl_0.size)
    # TODO: You can read the whole memory area rather then one element at a time and append
    predictions = []
    for index in range(index_lo, index_hi+1):
        ground_state_logit = mmio_bram.read(index*WORD_COUNT_PER_PREDICTION*WORD_SIZE_BYTE + 0)
        excited_state_logit = mmio_bram.read(index*WORD_COUNT_PER_PREDICTION*WORD_SIZE_BYTE + WORD_SIZE_BYTE)
        predictions.append([ground_state_logit, excited_state_logit])
    return predictions

def print_classifier_buffer(index_lo, index_hi):
    prediction_count = get_classifier_prediction_count()
    buffer_size = len(soccfg.axi_blk_bram_ctrl_0.mmio.array)
    
    print('INFO: prediction count: {:6}'.format(prediction_count))
    print('INFO: buffer size     : {:6}'.format(buffer_size))
    print('INFO:')
    for i in range(index_lo, index_hi+1):
        ground_state_logit, excited_state_logit = get_classifier_prediction(i)
        print('INFO: [{:5d}] {:08x} ({}) {}'.format(i, ground_state_logit, to_float(ground_state_logit), '<<<' if prediction_count == i else ''))
        print('INFO: [{:5d}] {:08x} ({}) {}'.format(i, excited_state_logit, to_float(excited_state_logit), '<<<' if prediction_count == i else ''))

# Basic loopback program calibration

## original code from demo

<!--- generator channel 6 : <-> Readout channel 0-->

In [ ]:
print('Generator channel:', GEN_CH)
print('Readout channel  :', RO_CH)

In [ ]:
class LoopbackProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        res_ch = cfg["res_ch"]

        # set the nyquist zone
        self.declare_gen(ch=cfg["res_ch"], nqz=1)
        
        # configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
        for ch in cfg["ro_chs"]:
            self.declare_readout(ch=ch, length=self.cfg["readout_length"],
                                 freq=self.cfg["pulse_freq"], gen_ch=cfg["res_ch"])

        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(cfg["pulse_freq"],gen_ch=res_ch, ro_ch=cfg["ro_chs"][0])
        phase = self.deg2reg(cfg["res_phase"], gen_ch=res_ch)
        gain = cfg["pulse_gain"]
        self.default_pulse_registers(ch=res_ch, freq=freq, phase=phase, gain=gain)

        style=self.cfg["pulse_style"]

        if style in ["flat_top","arb"]:
            sigma = cfg["sigma"]
            self.add_gauss(ch=res_ch, name="measure", sigma=sigma, length=sigma*5)
            
        if style == "const":
            self.set_pulse_registers(ch=res_ch, style=style, length=cfg["length"])
        elif style == "flat_top":
            # The first half of the waveform ramps up the pulse, the second half ramps down the pulse
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure", length=cfg["length"])
        elif style == "arb":
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure")
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        # fire the pulse
        # trigger all declared ADCs
        # pulse PMOD0_0 for a scope trigger
        # pause the tProc until readout is done
        # increment the time counter to give some time before the next measurement
        # (the syncdelay also lets the tProc get back ahead of the clock)
        self.measure(pulse_ch=self.cfg["res_ch"], 
                     adcs=self.ro_chs,
                     pins=[0], 
                     adc_trig_offset=self.cfg["adc_trig_offset"],
                     wait=True,
                     syncdelay=self.us2cycles(self.cfg["relax_delay"]))

In [ ]:
config={"res_ch": GEN_CH, #6, # --Fixed
        "ro_chs": [RO_CH], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":1.0, # --us
        "res_phase":0, # --degrees
        "pulse_style": "const", # --Fixed
        
        "length":20, # [Clock ticks]
        # Try varying length from 10-100 clock ticks
        
        "readout_length": 770, #100, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks

        "pulse_gain":3000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units

        "pulse_freq": 100, # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 100, # [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks

        "soft_avgs": 1 #100
        # Try varying soft_avgs from 1 to 200 averages

       }

###################
# Try it yourself !
###################

In [ ]:
prog = LoopbackProgram(soccfg, config)
iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=True)#, debug=False)

In [ ]:
# Plot results.
plt.figure(1)
for ii, iq in enumerate(iq_list):
    plt.plot(iq[0], label="I value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(iq[1], label="Q value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['ro_chs'][ii]))
plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
_ = plt.legend()
# plt.savefig("images/Send_recieve_pulse_const.pdf", dpi=350)

In [ ]:
print(iq_list)

## Time of flight (tof) calibration

Find the mis-alignment between the DAC and ADC readout window for previous experiment

In [ ]:
# Plot results.
plt.figure(1)
for ii, iq in enumerate(iq_list):
    plt.plot(iq[0], label="I value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(iq[1], label="Q value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['ro_chs'][ii]))
plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))

###########################################################
### using this 'cursor' to find the additional adc_trig_offset
axvline(64, ls = '--', color = 'grey', label = 'additional adc_trig_offset')
###########################################################

plt.legend()

Adjust the `adc_trig_offset` in the `config` dict

- The DAC and ADC has different fabric freq
    - DAC: fabric=430.080 MHz
    - ADC: fabric=307.200 MHz
- the readout plot data is sampled at ADC fabric freq, but the adc_trig_offset is used in DAC fabric freq
    - so we need to convert the measured delay into input delay:
    - input delay = meausured delay * DAC fabric freq/ADC fabric freq //1

In [ ]:
64*430.080/307.2

Now, the time of flight is calibrated

In [ ]:
config={"res_ch": GEN_CH, #6, # --Fixed
        "ro_chs": [RO_CH], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":1.0, # --us
        "res_phase":0, # --degrees
        "pulse_style": "const", # --Fixed
        
        "length":20, # [Clock ticks]
        # Try varying length from 10-100 clock ticks
        
        "readout_length": 770, #100, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks

        "pulse_gain":3000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units

        "pulse_freq": 100, # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        ###########################################################
        ## pervious was 100
        "adc_trig_offset": 190, # [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks
        ###########################################################

        "soft_avgs": 1, #100
        # Try varying soft_avgs from 1 to 200 averages

       }

###################
# Try it yourself !
###################

prog =LoopbackProgram(soccfg, config)
iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=True)#, debug=False)


###################
# plot the result
###################

# Plot results.
plt.figure(1)
for ii, iq in enumerate(iq_list):
    plt.plot(iq[0], label="I value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(iq[1], label="Q value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['ro_chs'][ii]))
plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))

axvline(63, ls = '--', color = 'grey', label = 'additional adc_trig_offset') ### using this 'cursor' to find the additional adc_trig_offset
plt.legend()

## Readout phase calibration
- code from qick_demos/05_PhaseCoherence_QickProgram.ipynb
- remeber to update the `adc_trig_offset` in the `config` file

In [ ]:
class LoopbackProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   

        # set the nyquist zone
        self.declare_gen(ch=cfg["res_ch"], nqz=1)

        self.r_rp=self.ch_page(self.cfg["res_ch"])     # get register page for res_ch
        self.r_gain=self.sreg(cfg["res_ch"], "gain")   #Get gain register for res_ch
        
        #configure the readout lengths and downconversion frequencies
        self.declare_readout(ch=cfg["ro_ch"], length=self.cfg["readout_length"],
                             freq=self.cfg["pulse_freq"], gen_ch=cfg["res_ch"])
        
        freq=self.freq2reg(cfg["pulse_freq"], gen_ch=cfg["res_ch"], ro_ch=cfg["ro_ch"])  # convert frequency to dac frequency (ensuring it is an available adc frequency)
        self.set_pulse_registers(ch=cfg["res_ch"], style="const", freq=freq, phase=0, gain=cfg["pulse_gain"], 
                                 length=cfg["length"])
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        self.measure(pulse_ch=self.cfg["res_ch"], 
             adcs=[self.cfg["ro_ch"]],
             adc_trig_offset=self.cfg["adc_trig_offset"],
             t=0,
             wait=True,
             syncdelay=self.us2cycles(self.cfg["relax_delay"]))

### First, sanity check that we can see the pulse we want to calibrate

In [ ]:
config={"res_ch": GEN_CH, #6, # --Fixed
        "ro_ch": RO_CH, # --Fixed
        "relax_delay":1.0, # --Fixed
        "res_phase":0, # --Fixed
        "length":400, # [Clock ticks] 
        "readout_length":770, #200, # [Clock ticks]
        "pulse_gain":10000, # [DAC units]
        "pulse_freq": 100, # [MHz]
        "adc_trig_offset": 190, # [Clock ticks]
        "reps":1, 
        "soft_avgs":1,
       }

prog = LoopbackProgram(soccfg, config)
(iq0,) = prog.acquire_decimated(soc,progress=False)

In [ ]:
# Plot results.
plt.figure(1)
plt.plot(iq0[0], label="I value; ADC 0")
plt.plot(iq0[1], label="Q value; ADC 0")
plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()

### Params 3 (We zoom in on the frequency area of interest and then print out the associated phase of interest)

In [ ]:
config={"res_ch": GEN_CH, #6, # --Fixed
        "ro_ch": RO_CH, # --Fixed
        "relax_delay":1.0, # --Fixed
        "res_phase":0, # --Fixed
        "length":400, # [Clock ticks] 
        "readout_length":770, #200, # [Clock ticks]
        "pulse_gain":10000, # [DAC units]
        "pulse_freq": 100, # [MHz]
        "adc_trig_offset": 190, # [Clock ticks]
        "reps":1, 
        "soft_avgs":1,
       }


freq_readout = config["pulse_freq"]
freq_start = freq_readout - 0.002 ##[MHz]
freq_step = 0.000125  ##[MHz]
expts = 32

sweep_cfg={"start":freq_start, "step":freq_step, "expts":expts}

gpts=sweep_cfg["start"] + sweep_cfg["step"]*np.arange(sweep_cfg["expts"])
gpts

In [ ]:
resultsi=[]
resultsq=[]
for g in gpts:
    time.sleep(0.1)
    config["pulse_freq"]=g
    prog =LoopbackProgram(soccfg, config)
    (iq0,) = prog.acquire_decimated(soc,progress=False)
    di0 = np.sum(iq0[0])/config["readout_length"]
    dq0 = np.sum(iq0[1])/config["readout_length"]
    resultsi.append(di0)
    resultsq.append(dq0)
resultsi=np.array(resultsi)
resultsq=np.array(resultsq)

In [ ]:
# Plot results.
sig = resultsi + 1j * resultsq
amp_array = np.abs(sig)
phase_array = np.angle(sig,deg=True)
for x in range(0,len(phase_array)):
    if phase_array[x] <0:
        phase_array[x] = phase_array[x] +360
    print("Iteration i = %d, freq_i = %f MHz, phi_i = %f degrees" %(x,gpts[x], phase_array[x]))
plt.figure(1)
# plt.plot(gpts, resultsi,label="I value; ADC 0")
# plt.plot(gpts, resultsq,label="Q value; ADC 0")
# plt.plot(gpts, amp_array,label="Amplitude (DAC units); ADC 0")
plt.plot(gpts, phase_array, label="Phase (degrees); ADC 0")
plt.plot(gpts,phase_array, marker='.', linestyle="None",color="Red")
plt.xticks(rotation=90)
plt.title(r"$\phi$ vs $f$")
plt.ylabel(r"$\phi$ (degrees)")
plt.xlabel(r"$f$ (MHz)")
plt.legend()
# plt.savefig("images/Phase_sweep.pdf", dpi=350)

In [ ]:
target_freq = freq_readout ### put the actual readout freq here

freq_index = np.where(gpts == target_freq)[0][0]
phase_cal = phase_array[freq_index]

print(f'At {target_freq} MHz, Phase = {phase_cal} degree')

plt.axvline(gpts[freq_index], ls = '--', color = 'grey')
plt.axhline(phase_array[freq_index], ls = '--', color = 'grey')


plt.plot(gpts, phase_array, label="Phase (degrees); ADC 0")
plt.plot(gpts,phase_array, marker='.', linestyle="None",color="Red")
plt.xticks(rotation=90)
plt.title(r"$\phi$ vs $f$")
plt.ylabel(r"$\phi$ (degrees)")
plt.xlabel(r"$f$ (MHz)")
plt.legend()

### re-run the loopback program with compensate phase
- Now the signal is only on I component

In [ ]:
class LoopbackProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        res_ch = cfg["res_ch"]

        # set the nyquist zone
        self.declare_gen(ch=cfg["res_ch"], nqz=1)
        
        # configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
        for ch in cfg["ro_chs"]:
            self.declare_readout(ch=ch, length=self.cfg["readout_length"],
                                 freq=self.cfg["pulse_freq"], gen_ch=cfg["res_ch"])

        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(cfg["pulse_freq"],gen_ch=res_ch, ro_ch=cfg["ro_chs"][0])
        phase = self.deg2reg(cfg["res_phase"], gen_ch=res_ch)
        gain = cfg["pulse_gain"]
        self.default_pulse_registers(ch=res_ch, freq=freq, phase=phase, gain=gain)

        style=self.cfg["pulse_style"]

        if style in ["flat_top","arb"]:
            sigma = cfg["sigma"]
            self.add_gauss(ch=res_ch, name="measure", sigma=sigma, length=sigma*5)
            
        if style == "const":
            self.set_pulse_registers(ch=res_ch, style=style, length=cfg["length"])
        elif style == "flat_top":
            # The first half of the waveform ramps up the pulse, the second half ramps down the pulse
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure", length=cfg["length"])
        elif style == "arb":
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure")
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        # fire the pulse
        # trigger all declared ADCs
        # pulse PMOD0_0 for a scope trigger
        # pause the tProc until readout is done
        # increment the time counter to give some time before the next measurement
        # (the syncdelay also lets the tProc get back ahead of the clock)
        self.measure(pulse_ch=self.cfg["res_ch"], 
                     adcs=self.ro_chs,
                     pins=[0], 
                     adc_trig_offset=self.cfg["adc_trig_offset"],
                     wait=True,
                     syncdelay=self.us2cycles(self.cfg["relax_delay"]))

In [ ]:
config={"res_ch": GEN_CH, #6, # --Fixed
        "ro_chs": [RO_CH], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":1.0, # --us
        "res_phase": phase_cal, # updated readout phase
        "pulse_style": "const", # --Fixed
        
        "length":20, # [Clock ticks]
        # Try varying length from 10-100 clock ticks
        
        "readout_length":770, #100, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks

        "pulse_gain":3000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units

        "pulse_freq": 100, # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 190, # [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks

        "soft_avgs": 1 #100
        # Try varying soft_avgs from 1 to 200 averages

       }

###################
# Try it yourself !
###################

prog =LoopbackProgram(soccfg, config)
iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=True)#, debug=False)

In [ ]:
# Plot results.
plt.figure(1)
for ii, iq in enumerate(iq_list):
    plt.plot(iq[0], label="I value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(iq[1], label="Q value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['ro_chs'][ii]))
plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()
# plt.savefig("images/Send_recieve_pulse_const.pdf", dpi=350)

# Load real qubit readout data (old data from ZCU111)


g_data[p1,p2,p3] for ground state readout

e_data[p1,p2,p3] for excited state readout
- p1: index for shot
- p2: index for I/Q
- p3: index for time, step of clock cycle = 2.6ns

In [ ]:
with open('readout_data.npy', 'rb') as f:
    g_data = np.load(f)
    e_data = np.load(f)

In [ ]:
# avargaed trace
plt.plot(g_data[:,1,:].mean(axis = 0), label = 'g_Q', color = 'b')
plt.plot(g_data[:,0,:].mean(axis = 0), label = 'g_I', color = 'b', ls = '--')

plt.plot(e_data[:,1,:].mean(axis = 0), label = 'e_Q', color = 'r')
plt.plot(e_data[:,0,:].mean(axis = 0), label = 'e_I', color = 'r', ls = '--')

plt.ylabel('ADC unit')
plt.xlabel('Time [clock cycle] ')
plt.legend()

In [ ]:
# IQ plot
I_g = g_data[:,0,:].mean(axis = 1)
Q_g = g_data[:,1,:].mean(axis = 1)

I_e = e_data[:,0,:].mean(axis = 1)
Q_e = e_data[:,1,:].mean(axis = 1)

plt.title('IQ Plot')
plt.plot(I_g, Q_g, '.',label = 'g',markersize = 5)
plt.plot(I_e, Q_e, '.',label = 'e',markersize = 5)
plt.xlabel('I')
plt.ylabel('Q')
plt.legend()
plt.axis('square')
plt.show()

vec_I = mean(I_e) - mean(I_g)
vec_Q = mean(Q_e) - mean(Q_g)

gstate = (I_g - mean(I_g))*vec_I + (Q_g - mean(Q_g))*vec_Q
estate = (I_e - mean(I_g))*vec_I + (Q_e - mean(Q_g))*vec_Q
gstate = gstate / abs(vec_I+1j*vec_Q)**2
estate = estate / abs(vec_I+1j*vec_Q)**2
plt.hist(gstate,bins = 100,label = 'g',alpha = 0.5)
plt.hist(estate,bins = 100,label = 'e',alpha = 0.5)
plt.legend()
plt.show()

th_min = min(gstate)
th_max = max(estate)
th_list = np.linspace(th_min,th_max,1000)

fidelity = [(sum(gstate<th)+sum(estate>th))/np.shape(gstate)[0]/2 for i,th in enumerate(th_list)]
print('max fidelity = ',max(fidelity)*2-1)

print('\n rough estimation of n_th')
100*sum(gstate>th_list[argmax(fidelity)])/len(gstate)

In [ ]:
# signal shot
shot_index = 0

plt.plot(g_data[shot_index,1,:], color = 'grey')
plt.plot(g_data[shot_index,0,:], color = 'grey', ls = '--')

plt.plot(e_data[shot_index,1,:], color = 'grey')
plt.plot(e_data[shot_index,0,:], color = 'grey', ls = '--')

plt.plot(g_data[:,1,:].mean(axis = 0), label = 'g_Q', color = 'b')
plt.plot(g_data[:,0,:].mean(axis = 0), label = 'g_I', color = 'b', ls = '--')

plt.plot(e_data[:,1,:].mean(axis = 0), label = 'e_Q', color = 'r')
plt.plot(e_data[:,0,:].mean(axis = 0), label = 'e_I', color = 'r', ls = '--')

plt.ylabel('ADC unit')
plt.xlabel('Time [clock cycle] ')
plt.legend()
# xlim(0,100)

# readout mock - averaged signal (as an example)

- to get the wf for DAC, we need to resample the data due to 
    - the frequency on ADC and DAC are different
    - the wf need to sample in dds frequency

            DAC tile 3, blk 2, 32-bit DDS, fabric=430.080 MHz, f_dds=6881.280 MHz
            ADC tile 2, blk 0, 32-bit DDS, fabric=307.200 MHz, fs=2457.600 MHz

In [ ]:
#### load real qubit readout data
data_len = len(g_data[:,0,:].mean(axis = 0))
print(f'real readout data length = {data_len} clock cycle')


### we need to resample the data based on DAC freq
t_ns = data_len*1000/307.2
dac_len = int((t_ns/1000*6881.28)//16*16)

g_I_load = g_data[:,0,:].mean(axis = 0)
g_Q_load = g_data[:,1,:].mean(axis = 0)
e_I_load = e_data[:,0,:].mean(axis = 0)
e_Q_load = e_data[:,1,:].mean(axis = 0)

### resample the waveform
xp = np.linspace(1,data_len,data_len) ### ADC side time array
x = np.linspace(1,data_len,dac_len)   ### DAC side time array

g_I = np.interp(x, xp, g_I_load)
g_Q = np.interp(x, xp, g_Q_load)
e_I = np.interp(x, xp, e_I_load)
e_Q = np.interp(x, xp, e_Q_load)


### plot one component of the resampled waveform
plot(x,g_I, 'o' , label = 'data load')
plot(xp,g_I_load, label = 'resample for DAC')
legend()
show()

In [ ]:
DAC_gain = 1 ### tune this parameter to match the ADC level with the training data

class LoopbackProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        res_ch = cfg["res_ch"]

        # set the nyquist zone
        self.declare_gen(ch=cfg["res_ch"], nqz=1)
        
        # configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
        for ch in cfg["ro_chs"]:
            self.declare_readout(ch=ch, length=self.cfg["readout_length"],
                                 freq=self.cfg["pulse_freq"], gen_ch=cfg["res_ch"])

        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(cfg["pulse_freq"],gen_ch=res_ch, ro_ch=cfg["ro_chs"][0])
        phase = self.deg2reg(cfg["res_phase"], gen_ch=res_ch)
        gain = cfg["pulse_gain"]
        self.default_pulse_registers(ch=res_ch, freq=freq, phase=phase, gain=gain)

        style=self.cfg["pulse_style"]
        
        
        ########################
        ### add g/e waveform ###
        ########################
#         self.add_pulse(ch=cfg["res_ch"], name = 'g_state', idata = DAC_gain*g_I, qdata = DAC_gain*g_Q)
#         self.add_pulse(ch=cfg["res_ch"], name = 'e_state', idata = DAC_gain*e_I, qdata = DAC_gain*e_Q)
        self.add_pulse(ch=cfg["res_ch"], name = 'g_state', idata = DAC_gain*g_Q, qdata = DAC_gain*g_I)
        self.add_pulse(ch=cfg["res_ch"], name = 'e_state', idata = DAC_gain*e_Q, qdata = DAC_gain*e_I)

        if style in ["flat_top","arb"]:
            sigma = cfg["sigma"]
            self.add_gauss(ch=res_ch, name="measure", sigma=sigma, length=sigma*5)
            
        if style == "const":
            self.set_pulse_registers(ch=res_ch, style=style, length=cfg["length"])
        elif style == "flat_top":
            # The first half of the waveform ramps up the pulse, the second half ramps down the pulse
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure", length=cfg["length"])
        elif style == "arb":
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure")
            
        ########################
        ### add g/e waveform ###
        ########################   
        elif style == "g_state":
            self.set_pulse_registers(ch=res_ch, style="arb", waveform="g_state")
        elif style == "e_state":
            self.set_pulse_registers(ch=res_ch, style="arb", waveform="e_state")

        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        # fire the pulse
        # trigger all declared ADCs
        # pulse PMOD0_0 for a scope trigger
        # pause the tProc until readout is done
        # increment the time counter to give some time before the next measurement
        # (the syncdelay also lets the tProc get back ahead of the clock)
        self.measure(pulse_ch=self.cfg["res_ch"], 
                     adcs=self.ro_chs,
                     pins=[0], 
                     adc_trig_offset=self.cfg["adc_trig_offset"],
                     wait=True,
                     syncdelay=self.us2cycles(self.cfg["relax_delay"]))

In [ ]:
### calculate the DAC output range

gencfg = soccfg['gens'][GEN_CH]
max_amp = gencfg['maxv']*gencfg['maxv_scale']
print('max_amp = ',max_amp)

In [ ]:
DAC_phase = -90
readout_phase = phase_cal + DAC_phase

config={"res_ch": GEN_CH, #6, # --Fixed
        "ro_chs": [RO_CH], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":1.0, # --us
        "res_phase": readout_phase, # updated readout phase
        "pulse_style": "e_state", # --Fixed,"const"; "g_state"; "e_state"; "arb"
        
        "sigma": 30, # [Clock ticks]
        
        "length": 100, # [Clock ticks]
        # Try varying length from 10-100 clock ticks
        
        "readout_length":data_len, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks

        "pulse_gain": 30000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units

        "pulse_freq": 100, # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 190, # [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks

        "soft_avgs":1
        # Try varying soft_avgs from 1 to 200 averages

       }

###################
# Try it yourself !
###################

prog =LoopbackProgram(soccfg, config)
iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=True)#, debug=False)

In [ ]:
# Plot results.
plt.figure(1)
for ii, iq in enumerate(iq_list):
    plt.plot(iq[0], label="I value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(iq[1], label="Q value, ADC %d"%(config['ro_chs'][ii]))
#     plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['ro_chs'][ii]))
plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()
# plt.savefig("images/Send_recieve_pulse_const.pdf", dpi=350)

In [ ]:
ADC_fake_gain = 27 ### tune this parameter to match the ADC level with the training data


for ii, iq in enumerate(iq_list):
    plt.plot(iq[0]*ADC_fake_gain, markersize=0.5, color = 'C0' , label="I value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(iq[1]*ADC_fake_gain, markersize=0.5, color = 'C1' ,label="Q value, ADC %d"%(config['ro_chs'][ii]))
plt.plot(x,e_I, '-o', markersize=0.5, color = 'grey', label = 'load - e-I')
plt.plot(x,e_Q, '-o', markersize=0.5, color = 'black', label = 'load - e-Q')


plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()

# Readout mock - single-shot signal

## single-shot with tunable gaussian noise

In [ ]:
MEAN_ENABLED = False
TRACE_ID = 0

In [ ]:
#### load real qubit readout data
data_len = len(g_data[:,0,:].mean(axis = 0))
print(f'readout data length = {data_len} clock cycle')

### we need to resample the data based on DAC freq
t_ns = data_len*1000/307.2
dac_len = int((t_ns/1000*6881.28)//16*16)

if MEAN_ENABLED:
    g_I_load = g_data[:,0,:].mean(axis = 0)
    g_Q_load = g_data[:,1,:].mean(axis = 0)
    e_I_load = e_data[:,0,:].mean(axis = 0)
    e_Q_load = e_data[:,1,:].mean(axis = 0)
else:
    g_I_load = g_data[TRACE_ID,0,:]
    g_Q_load = g_data[TRACE_ID,1,:]
    e_I_load = e_data[TRACE_ID,0,:]
    e_Q_load = e_data[TRACE_ID,1,:]

### set noise parameter
add_noise = False
noise_scale = 400

if add_noise and MEAN_ENABLED:
    g_I_add_noise = g_I_load + np.random.normal(0, noise_scale, size = g_I_load.shape[0])
    g_Q_add_noise = g_Q_load + np.random.normal(0, noise_scale, size = g_Q_load.shape[0])
    e_I_add_noise = e_I_load + np.random.normal(0, noise_scale, size = e_I_load.shape[0])
    e_Q_add_noise = e_Q_load + np.random.normal(0, noise_scale, size = e_Q_load.shape[0])
else:
    g_I_add_noise = g_I_load
    g_Q_add_noise = g_Q_load
    e_I_add_noise = e_I_load
    e_Q_add_noise = e_Q_load
    
plot(xp,g_I_load , label = 'data load')
plot(xp,g_I_add_noise, label = 'data load + noise', alpha = 0.4)
title('data vs data + noise')
legend()
show()
### resample the waveform
xp = np.linspace(1,data_len,data_len)
x = np.linspace(1,data_len,dac_len)
    
g_I = np.interp(x, xp, g_I_load)
g_Q = np.interp(x, xp, g_Q_load)
e_I = np.interp(x, xp, e_I_load)
e_Q = np.interp(x, xp, e_Q_load)

if add_noise and MEAN_ENABLED:
    g_I = np.interp(x, xp, g_I_add_noise)
    g_Q = np.interp(x, xp, g_Q_add_noise)
    e_I = np.interp(x, xp, e_I_add_noise)
    e_Q = np.interp(x, xp, e_Q_add_noise)
    
    title('Performance of the resampling')
    plot(x, g_I, 'o', label = 'after resampling')
    plot(xp, g_I_add_noise, label = 'before resampling')
    legend()
    show()

In [ ]:
DAC_gain = 1 ### tune this parameter to match the ADC level with the training data

class LoopbackProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        res_ch = cfg["res_ch"]

        # set the nyquist zone
        self.declare_gen(ch=cfg["res_ch"], nqz=1)
        
        # configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
        for ch in cfg["ro_chs"]:
            self.declare_readout(ch=ch, length=self.cfg["readout_length"],
                                 freq=self.cfg["pulse_freq"], gen_ch=cfg["res_ch"])

        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(cfg["pulse_freq"],gen_ch=res_ch, ro_ch=cfg["ro_chs"][0])
        phase = self.deg2reg(cfg["res_phase"], gen_ch=res_ch)
        gain = cfg["pulse_gain"]
        self.default_pulse_registers(ch=res_ch, freq=freq, phase=phase, gain=gain)

        style=self.cfg["pulse_style"]
        
        
        ########################
        ### add g/e waveform ###
        ########################
#         self.add_pulse(ch=cfg["res_ch"], name = 'g_state', idata = DAC_gain*g_I, qdata = DAC_gain*g_Q)
#         self.add_pulse(ch=cfg["res_ch"], name = 'e_state', idata = DAC_gain*e_I, qdata = DAC_gain*e_Q)
        self.add_pulse(ch=cfg["res_ch"], name = 'g_state', idata = DAC_gain*g_Q, qdata = DAC_gain*g_I)
        self.add_pulse(ch=cfg["res_ch"], name = 'e_state', idata = DAC_gain*e_Q, qdata = DAC_gain*e_I)

        if style in ["flat_top","arb"]:
            sigma = cfg["sigma"]
            self.add_gauss(ch=res_ch, name="measure", sigma=sigma, length=sigma*5)
            
        if style == "const":
            self.set_pulse_registers(ch=res_ch, style=style, length=cfg["length"])
        elif style == "flat_top":
            # The first half of the waveform ramps up the pulse, the second half ramps down the pulse
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure", length=cfg["length"])
        elif style == "arb":
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure")
            
        ########################
        ### add g/e waveform ###
        ########################   
        elif style == "g_state":
            self.set_pulse_registers(ch=res_ch, style="arb", waveform="g_state")
        elif style == "e_state":
            self.set_pulse_registers(ch=res_ch, style="arb", waveform="e_state")

        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        # fire the pulse
        # trigger all declared ADCs
        # pulse PMOD0_0 for a scope trigger
        # pause the tProc until readout is done
        # increment the time counter to give some time before the next measurement
        # (the syncdelay also lets the tProc get back ahead of the clock)
        self.measure(pulse_ch=self.cfg["res_ch"], 
                     adcs=self.ro_chs,
                     pins=[0], 
                     adc_trig_offset=self.cfg["adc_trig_offset"],
                     wait=True,
                     syncdelay=self.us2cycles(self.cfg["relax_delay"]))

In [ ]:
### calculate the DAC output range
gencfg = soccfg['gens'][GEN_CH]
max_amp = gencfg['maxv']*gencfg['maxv_scale']
print('max_amp = ',max_amp)

In [ ]:
##########################################################
##########################################################
##########################################################
# If you are running Vivado ILAs, it is time to arm them #
##########################################################
##########################################################
##########################################################

In [ ]:
# How many readouts so far?
get_classifier_prediction_count()

In [ ]:
%%time
if HAS_NN:
    SCALING_FACTOR = ADC_fake_gain
    TRIGGER_DELAY = 0
    reset_classifier(deep_reset = True, index_lo = 0, index_hi = 63)
    configure_classifier(SCALING_FACTOR, TRIGGER_DELAY)

In [ ]:
# 0 readouts after reset
get_classifier_prediction_count()

In [ ]:
DAC_phase = -90
readout_phase = phase_cal + DAC_phase

config={"res_ch": GEN_CH, #6, # --Fixed
        "ro_chs": [RO_CH], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":1.0, # --us
        "res_phase": readout_phase, # updated readout phase
        
        "pulse_style": "e_state", # --Fixed,"const"; "g_state"; "e_state"; "arb"
        #"pulse_style": "g_state", # --Fixed,"const"; "g_state"; "e_state"; "arb"
        
        "sigma": 30, # [Clock ticks]
        
        "length": 100, # [Clock ticks]
        # Try varying length from 10-100 clock ticks
        
        "readout_length":data_len, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks

        "pulse_gain": 30000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units

        "pulse_freq": 100, # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 190, # [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks

        "soft_avgs":1
        # Try varying soft_avgs from 1 to 200 averages

       }

###################
# Try it yourself !
###################

prog = LoopbackProgram(soccfg, config)
iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=True)#, debug=False)

In [ ]:
iq_list.shape

In [ ]:
# Plot results.
plt.figure(1)
for ii, iq in enumerate(iq_list):
    plt.plot(iq[0], label="I value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(iq[1], label="Q value, ADC %d"%(config['ro_chs'][ii]))
#     plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['ro_chs'][ii]))
plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()
# plt.savefig("images/Send_recieve_pulse_const.pdf", dpi=350)

In [ ]:
print('iq_list shape (I/Q):', iq_list[0].shape)
print('data_len:', data_len)

# Shows I and Q sequence
iq = iq_list[0]
iq_sequence = [list(item) for item in zip(*iq)]
iq_sequence = [int(item) for sublist in iq_sequence for item in sublist]
print(iq_sequence)

In [ ]:
ADC_fake_gain = 27 ### tune this parameter to match the ADC level with the training data

plt.plot(x,e_I, '-o', markersize=1.5, color = 'grey', label = 'load - e-I', alpha = 0.5)
plt.plot(x,e_Q, '-o', markersize=1.5, color = 'black', label = 'load - e-Q', alpha = 0.5)

for ii, iq in enumerate(iq_list):
    plt.plot(iq[0]*ADC_fake_gain, markersize=0.5, color = 'C0' , label="I value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(iq[1]*ADC_fake_gain, markersize=0.5, color = 'C1' ,label="Q value, ADC %d"%(config['ro_chs'][ii]))

plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()

## Collect NN results

In [ ]:
###########################################################
### NN input window = 100 couples I/Q starting at index 285
axvline(285, ls = '--', color = 'red')
axvline(285+100, ls = '--', color = 'red')
###########################################################

#plt.plot(x,e_I, '-o', markersize=1.5, color = 'grey', label = 'load - e-I', alpha = 0.5)
#plt.plot(x,e_Q, '-o', markersize=1.5, color = 'black', label = 'load - e-Q', alpha = 0.5)
for ii, iq in enumerate(iq_list):
    plt.plot(iq[0]*ADC_fake_gain, markersize=0.5, color = 'C0' , label="I value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(iq[1]*ADC_fake_gain, markersize=0.5, color = 'C1' ,label="Q value, ADC %d"%(config['ro_chs'][ii]))

plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()

In [ ]:
# Save trace.
iq_nn = iq_list[0,:,285:385]*ADC_fake_gain

In [ ]:
import struct
def float_to_hex32(f):
    return format(struct.unpack('!I', struct.pack('!f', f))[0], '08x')

def int_to_twos_complement_hex32(n):
    # If the number is negative, get its two's complement
    if n < 0:
        n = (1 << 32) + n  # "Wrap around" to get 32-bit two's complement
    return format(n, '08x')

# Shows I and Q sequence
iq_sequence_nn = [list(item) for item in zip(*iq_nn)]
iq_sequence_nn = [int(item) for sublist in iq_sequence_nn for item in sublist]

# Float value as hex
#hex_iq_sequence_flt = [float_to_hex32(num) for num in iq_sequence]

# Float -> Int value as hex
#hex_iq_sequence_dec = [int_to_twos_complement_hex32(int(i)) for i in iq_sequence]

print('Sequence length:', len(iq_sequence_nn))
print('I (lo, data[15:0])')
print(iq_nn[0])
print('Q (hi, data[31:16])')
print(iq_nn[1])

print(iq_sequence_nn)

#print(hex_iq_sequence_flt)
#print(hex_iq_sequence_dec)

#for i in range(0,len(iq_sequence),2):
#    print("I {:4.0f} {} {}".format(iq_sequence[i], hex_iq_sequence_flt[i], hex_iq_sequence_dec[i]))
#    print("Q {:4.0f} {} {}".format(iq_sequence[i+1], hex_iq_sequence_flt[i+1], hex_iq_sequence_dec[i+1]))

In [ ]:
plt.plot(iq_nn[0], markersize=0.5, color = 'C0', label="I value")
plt.plot(iq_nn[1], markersize=0.5, color = 'C1', label="Q value")

# x = list(range(0, 101, 20))
# x_offset = list(range(285, 386, 20))
# plt.xticks(x, x_offset)

plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()

In [ ]:
#
# ATTENTION: if the cell fails, please make sure that
#            the base_address matches the associated
#            value in the address editor of the Vivado
#            project.
# 
if HAS_NN:    
    from ctypes import *
    def to_float(i):
        cp = pointer(c_int(i))
        fp = cast(cp, POINTER(c_float))
        return fp.contents.value
    
    ground_state_logit, excited_state_logit = get_classifier_prediction()

    if config['pulse_style'] == 'g_state':
        print('Expected:   ground state')
    else:
        print('Expected:   excited state')
    
    if (to_float(ground_state_logit) > to_float(excited_state_logit)):
        print('Prediction: ground state', end='')
        if config['pulse_style'] == 'g_state':
            print(', CORRECT')
        else:
            print(', WRONG')
    else:
        print('Prediction: excited state', end='')
        if config['pulse_style'] == 'e_state':
            print(', CORRECT')
        else:
            print(', WRONG')
    print('')
    print('Logit values as int: [', ground_state_logit, ',', excited_state_logit, ']')
    print('Logit values as hex: [', hex(ground_state_logit), ',', hex(excited_state_logit), ']')
    print('Logit values as flt: [', to_float(ground_state_logit), ',', to_float(excited_state_logit), ']')
else:
    print('No NN in this bitstream!')

In [ ]:
print_classifier_buffer(index_lo = 0, index_hi = 31)

In [ ]:
#reset_classifier(deep_reset = False, index_lo = 0, index_hi = 0)

In [ ]:
#print_classifier_buffer(0,7)

## Many runs and collect results

In [ ]:
# # Uncomment the following lines to cleanup collected results
# !rm -f malab_e_state_A_fpga.csv  malab_g_state_A_fpga.csv
# !rm -f malab_e_state_X_fpga.csv  malab_e_state_y_fpga.csv
# !rm -f malab_g_state_X_fpga.csv  malab_g_state_y_fpga.csv

In [ ]:
# How many readouts so far?
get_classifier_prediction_count()

In [ ]:
%%time
if HAS_NN:
    SCALING_FACTOR = ADC_fake_gain
    TRIGGER_DELAY = 0
    reset_classifier(deep_reset = True, index_lo = 0, index_hi = 63)
    configure_classifier(SCALING_FACTOR, TRIGGER_DELAY)

In [ ]:
# 0 readouts after reset
get_classifier_prediction_count()

In [ ]:
VERBOSE = False

In [ ]:
%%time
import csv
import pandas as pd

from ctypes import *

errors = {'g_state' : 0, 'e_state' : 0}

def to_float(i):
    cp = pointer(c_int(i))
    fp = cast(cp, POINTER(c_float))
    return fp.contents.value

### load real qubit readout data
data_len = len(g_data[0,0,:])
if VERBOSE:
    print(f'readout data length = {data_len} clock cycle')

### how many traces we have?
trace_count = {'g_state' : len(g_data[:,0,:]), 'e_state' : len(e_data[:,0,:])}

if VERBOSE:
    print('ground traces count  = {}'.format(trace_count['g_state']))
    print('excited traces count = {}'.format(trace_count['e_state']))

### we need to resample the data based on DAC freq
t_ns = data_len*1000/307.2
dac_len = int((t_ns/1000*6881.28)//16*16)


if HAS_NN:

    ################
    # Configure NN #
    ################
    configure_classifier(scaling_factor=ADC_fake_gain, trigger_delay=5)
    ################

    for state_id, pulse_style in enumerate(['g_state', 'e_state']):
        
        #n_traces = trace_count[pulse_style]
        n_traces = 2000
        
        if not VERBOSE:
            print('-' * 80)
            print('Sending {} {} traces to the NN'.format(n_traces, pulse_style))
            print('Legend:')
            print('  . = CORRECT classification')
            print('  * = WRONG   classification')

        DAC_phase = -90
        readout_phase = phase_cal + DAC_phase

        config={"res_ch": GEN_CH, #6, # --Fixed
                "ro_chs": [RO_CH], # --Fixed
                "reps":1, # --Fixed
                "relax_delay":1.0, # --us
                "res_phase": readout_phase, # updated readout phase

                "pulse_style": pulse_style, # --Fixed,"const"; "g_state"; "e_state"; "arb"

                "sigma": 30, # [Clock ticks]

                "length": 100, # [Clock ticks]
                # Try varying length from 10-100 clock ticks

                "readout_length":data_len, # [Clock ticks]
                # Try varying readout_length from 50-1000 clock ticks

                "pulse_gain": 30000, # [DAC units]
                # Try varying pulse_gain from 500 to 30000 DAC units

                "pulse_freq": 100, # [MHz]
                # In this program the signal is up and downconverted digitally so you won't see any frequency
                # components in the I/Q traces below. But since the signal gain depends on frequency, 
                # if you lower pulse_freq you will see an increased gain.

                "adc_trig_offset": 190, # [Clock ticks]
                # Try varying adc_trig_offset from 100 to 220 clock ticks

                "soft_avgs":1
                # Try varying soft_avgs from 1 to 200 averages

               }        
        
        for trace_id in range(n_traces):
            g_I_load = g_data[trace_id,0,:]
            g_Q_load = g_data[trace_id,1,:]
            e_I_load = e_data[trace_id,0,:]
            e_Q_load = e_data[trace_id,1,:]

            g_I = np.interp(x, xp, g_I_load)
            g_Q = np.interp(x, xp, g_Q_load)
            e_I = np.interp(x, xp, e_I_load)
            e_Q = np.interp(x, xp, e_Q_load)

            DAC_gain = 1 ### tune this parameter to match the ADC level with the training data

            class LoopbackProgram(AveragerProgram):
                def initialize(self):
                    cfg = self.cfg   
                    res_ch = cfg["res_ch"]

                    # set the nyquist zone
                    self.declare_gen(ch=cfg["res_ch"], nqz=1)

                    # configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
                    for ch in cfg["ro_chs"]:
                        self.declare_readout(ch=ch, length=self.cfg["readout_length"],
                                             freq=self.cfg["pulse_freq"], gen_ch=cfg["res_ch"])

                    # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
                    freq = self.freq2reg(cfg["pulse_freq"],gen_ch=res_ch, ro_ch=cfg["ro_chs"][0])
                    phase = self.deg2reg(cfg["res_phase"], gen_ch=res_ch)
                    gain = cfg["pulse_gain"]
                    self.default_pulse_registers(ch=res_ch, freq=freq, phase=phase, gain=gain)

                    style=self.cfg["pulse_style"]


                    ########################
                    ### add g/e waveform ###
                    ########################
                    #self.add_pulse(ch=cfg["res_ch"], name = 'g_state', idata = DAC_gain*g_I, qdata = DAC_gain*g_Q)
                    #self.add_pulse(ch=cfg["res_ch"], name = 'e_state', idata = DAC_gain*e_I, qdata = DAC_gain*e_Q)
                    self.add_pulse(ch=cfg["res_ch"], name = 'g_state', idata = DAC_gain*g_Q, qdata = DAC_gain*g_I)
                    self.add_pulse(ch=cfg["res_ch"], name = 'e_state', idata = DAC_gain*e_Q, qdata = DAC_gain*e_I)

                    if style in ["flat_top","arb"]:
                        sigma = cfg["sigma"]
                        self.add_gauss(ch=res_ch, name="measure", sigma=sigma, length=sigma*5)

                    if style == "const":
                        self.set_pulse_registers(ch=res_ch, style=style, length=cfg["length"])
                    elif style == "flat_top":
                        # The first half of the waveform ramps up the pulse, the second half ramps down the pulse
                        self.set_pulse_registers(ch=res_ch, style=style, waveform="measure", length=cfg["length"])
                    elif style == "arb":
                        self.set_pulse_registers(ch=res_ch, style=style, waveform="measure")

                    ########################
                    ### add g/e waveform ###
                    ########################   
                    elif style == "g_state":
                        self.set_pulse_registers(ch=res_ch, style="arb", waveform="g_state")
                    elif style == "e_state":
                        self.set_pulse_registers(ch=res_ch, style="arb", waveform="e_state")


                    self.synci(200)  # give processor some time to configure pulses

                def body(self):
                    # fire the pulse
                    # trigger all declared ADCs
                    # pulse PMOD0_0 for a scope trigger
                    # pause the tProc until readout is done
                    # increment the time counter to give some time before the next measurement
                    # (the syncdelay also lets the tProc get back ahead of the clock)
                    self.measure(pulse_ch=self.cfg["res_ch"], 
                                 adcs=self.ro_chs,
                                 pins=[0], 
                                 adc_trig_offset=self.cfg["adc_trig_offset"],
                                 wait=True,
                                 syncdelay=self.us2cycles(self.cfg["relax_delay"]))

            # calculate the DAC output range
            gencfg = soccfg['gens'][GEN_CH]
            max_amp = gencfg['maxv']*gencfg['maxv_scale']
            #print('max_amp = ',max_amp)

            # Run pulse generation and readout
            # It also runs the NN
            prog = LoopbackProgram(soccfg, config)
            iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=VERBOSE)#, debug=False)

            ###################
            # Read NN results #
            ###################
            ground_state_logit, excited_state_logit = get_classifier_prediction(state_id * n_traces + trace_id)
            ###################

            # Console printous
            if VERBOSE:
                if pulse_style == 'g_state':
                    print('Expected:   ground state')
                else:
                    print('Expected:   excited state')

                print('{} {}'.format(to_float(ground_state_logit), to_float(excited_state_logit)))
                if (to_float(ground_state_logit) > to_float(excited_state_logit)):
                    print('Prediction: ground state', end='')
                    if pulse_style == 'g_state':
                        print(', CORRECT')
                    else:
                        print(', WRONG')
                else:
                    print('Prediction: excited state', end='')
                    if pulse_style == 'e_state':
                        print(', CORRECT')
                    else:
                        print(', WRONG')
                print('')
                print('Logit values as int: [', ground_state_logit, ',', excited_state_logit, ']')
                print('Logit values as hex: [', hex(ground_state_logit), ',', hex(excited_state_logit), ']')
                print('Logit values as flt: [', to_float(ground_state_logit), ',', to_float(excited_state_logit), ']')

            # Error count
            if (to_float(ground_state_logit) > to_float(excited_state_logit)):
                if pulse_style != 'g_state':
                    errors[pulse_style] = errors[pulse_style] + 1
            else:
                if pulse_style != 'e_state':
                    errors[pulse_style] = errors[pulse_style] + 1

            if not VERBOSE:
                if (trace_id % 25) == 0:
                    print('{:s}{:5d}'.format(pulse_style, trace_id), end=' ')
                if (to_float(ground_state_logit) > to_float(excited_state_logit)):
                    if pulse_style != 'g_state':
                        print('*', end='')
                    else:
                        print('.', end='')
                else:
                    if pulse_style != 'e_state':
                        print('*', end='')
                    else:
                        print('.', end='')
                if ((trace_id+1) % 25) == 0:
                    print('')

            if SAVE_NN_TRACES:  
                iq_nn = iq_list[0,:,285:385]*ADC_fake_gain
                iq_sequence_nn = [list(item) for item in zip(*iq_nn)]
                iq_sequence_nn = [int(item) for sublist in iq_sequence_nn for item in sublist]

                iq = iq_list[0,:,:]*ADC_fake_gain
                iq_sequence = [list(item) for item in zip(*iq)]
                iq_sequence = [int(item) for sublist in iq_sequence for item in sublist]

                # Save on file I/Q trace and NN prediction
                A_fpga_csv_file = 'malab_{}_A_fpga.csv'.format(pulse_style)
#                 X_fpga_csv_file = 'malab_{}_X_fpga.csv'.format(pulse_style)
#                 y_fpga_csv_file = 'malab_{}_y_fpga.csv'.format(pulse_style)

                with open(A_fpga_csv_file, mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerows([iq_sequence])

#                 with open(X_fpga_csv_file, mode='a', newline='') as file:
#                     writer = csv.writer(file)
#                     writer.writerows([iq_sequence_nn])

#                 with open(y_fpga_csv_file, mode='a', newline='') as file:
#                     writer = csv.writer(file)
#                     writer.writerows([[ground_state_logit, excited_state_logit]])
else:
    print('No NN in this bitstream!')

In [ ]:
print_classifier_buffer(index_lo=0, index_hi=7)

In [ ]:
print_classifier_buffer(index_lo=100, index_hi=107)

In [ ]:
# Show error rate
total_errors = errors['g_state'] + errors['e_state']
total_traces = trace_count['g_state'] + trace_count['e_state']
print('--- Total ---')
print('Errors: {}/{}'.format(total_errors, total_traces))
print('Error rate  : {:.4}%'.format(100. * total_errors / total_traces))
print('Accuracy    : {:.4}%'.format(100. - (100. * total_errors / total_traces)))
print('--- Ground state ---')
print('Errors: {}/{}'.format(errors['g_state'], trace_count['g_state']))
print('Error rate  : {:.4}%'.format(100. * errors['g_state'] / trace_count['g_state']))
print('Accuracy    : {:.4}%'.format(100. - (100. * errors['g_state'] / trace_count['g_state'])))
print('--- Excited state ---')
print('Errors: {}/{}'.format(errors['e_state'], trace_count['e_state']))
print('Error rate  : {:.4}%'.format(100. * errors['e_state'] / trace_count['e_state']))
print('Accuracy    : {:.4}%'.format(100. - (100. * errors['e_state'] / trace_count['e_state'])))

In [ ]:
if HAS_NN:
    print('{:6d}|{:6d}|{:3.2f}%|{:3.2f}%|{:6d}|{:6d}|{:3.2f}%|{:3.2f}%|{:6d}|{:6d}|{:3.2f}%|{:3.2f}%'.format(
          total_errors,
          total_traces,
          100. * total_errors / total_traces, 
          100. - (100. * total_errors / total_traces),
          errors['g_state'],
          trace_count['g_state'],
          100. * errors['g_state'] / trace_count['g_state'],
          100. - (100. * errors['g_state'] / trace_count['g_state']),
          errors['e_state'],
          trace_count['e_state'],
          100. * errors['e_state'] / trace_count['e_state'],
          100. - (100. * errors['e_state'] / trace_count['e_state'])
         ))

 ```
 770 | 3149| 10000|31.49%|68.51%|   253|  5000| 5.06%|94.94%|  2896|  5000|57.92%|42.08%
 760 | 4425| 10000|44.25%|55.75%|    24|  5000| 0.48%|99.52%|  4401|  5000|88.02%|11.98%
 400 |  440| 10000| 4.40%|95.60%|   109|  5000| 2.18%|97.82%|   331|  5000| 6.62%|93.38%
 200 |  696| 10000| 6.96%|93.04%|   241|  5000| 4.82%|95.18%|   455|  5000| 9.10%|90.90%
 100 | 1392| 10000|13.92%|86.08%|   539|  5000|10.78%|89.22%|   853|  5000|17.06%|82.94%
 ```

## Average in python